In [0]:
%pip install pillow

In [0]:
# ======================================================
# JOB: Carga de áreas queimadas (Arquivo TIFF)
# ======================================================

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from PIL import Image
import numpy as np
import os

spark = SparkSession.builder.getOrCreate()

In [0]:
# ======================================================
# 1️⃣ - Parâmetro de entrada
# ======================================================
dbutils.widgets.text("data_referencia", "")
referencia = dbutils.widgets.get("data_referencia").strip()
# Exemplo local:
# referencia = "202509"

if not referencia or len(referencia) != 6:
    raise ValueError("⚠️ Informe a data_referencia no formato AAAAMM (ex: 202509).")

print(f"📅 Data de referência: {referencia}")

In [0]:
# ======================================================
# 2️⃣ - Caminho e nome do arquivo
# ======================================================
origem_base = "/Volumes/datamasters/raw/raw_inpe"
nome_arquivo = f"aqm1km_{referencia}.tif"
caminho_arquivo = os.path.join(origem_base, nome_arquivo)

print(f"📂 Procurando arquivo: {caminho_arquivo}")

if not any(nome_arquivo in f.path for f in dbutils.fs.ls(origem_base)):
    raise FileNotFoundError(f"🚫 Arquivo {nome_arquivo} não encontrado em {origem_base}")

In [0]:
# ======================================================
# 3️⃣ - Leitura do TIFF com Pillow
# ======================================================
img = Image.open(caminho_arquivo)
array = np.array(img)

print(f"📏 Dimensões da imagem: {array.shape}")

In [0]:
# ======================================================
# 4️⃣ - Conversão em DataFrame Spark
# ======================================================
# (Transforma pixels em linhas — pode ser pesado em imagens grandes)
linhas = array.shape[0]
colunas = array.shape[1]

# Gera tuplas (linha, coluna, valor)
dados = [(int(i), int(j), float(array[i, j])) for i in range(linhas) for j in range(colunas)]

df = spark.createDataFrame(dados, ["linha", "coluna", "valor"])

df_final = (
    df.withColumn("data_referencia", F.lit(referencia))
      .withColumn("data_carga", F.current_timestamp())
)

display(df_final.limit(5))

In [0]:
# ======================================================
# 5️⃣ - Gravação em Delta Lake
# ======================================================
tabela_delta = "datamasters.b_inep.m_area_queim"

(
    df_final.write
    .format("delta")
    .mode("append")
    .partitionBy("data_referencia")
    .saveAsTable(tabela_delta)
)

print(f"✅ Dados gravados com sucesso na tabela {tabela_delta}")